In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns
from typing import List, Tuple, Dict, Optional
import time
import warnings
from collections import OrderedDict
from google.colab import files
import random


In [ ]:
AMINO_ACIDS = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L',
               'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']

AA_TO_IDX = {aa: idx + 1 for idx, aa in enumerate(AMINO_ACIDS)}
AA_TO_IDX['X'] = 0

# Physicochemical properties (normalized)
AA_PROPERTIES = {
    # Hydrophobicity (Kyte-Doolittle scale, normalized to [-1, 1])
    'hydrophobicity': {
        'A': 0.47, 'C': 0.67, 'D': -0.93, 'E': -0.93, 'F': 0.73,
        'G': -0.10, 'H': -0.80, 'I': 1.13, 'K': -0.93, 'L': 0.97,
        'M': 0.53, 'N': -0.93, 'P': -0.47, 'Q': -0.93, 'R': -1.20,
        'S': -0.20, 'T': -0.17, 'V': 1.07, 'W': -0.27, 'Y': -0.40, 'X': 0.0
    },
    # Charge at pH 7
    'charge': {
        'A': 0, 'C': 0, 'D': -1, 'E': -1, 'F': 0,
        'G': 0, 'H': 0.1, 'I': 0, 'K': 1, 'L': 0,
        'M': 0, 'N': 0, 'P': 0, 'Q': 0, 'R': 1,
        'S': 0, 'T': 0, 'V': 0, 'W': 0, 'Y': 0, 'X': 0
    },
    # Polarity
    'polarity': {
        'A': 0, 'C': 1, 'D': 1, 'E': 1, 'F': 0,
        'G': 0, 'H': 1, 'I': 0, 'K': 1, 'L': 0,
        'M': 0, 'N': 1, 'P': 0, 'Q': 1, 'R': 1,
        'S': 1, 'T': 1, 'V': 0, 'W': 0, 'Y': 1, 'X': 0
    },
    # Size (molecular weight, normalized)
    'size': {
        'A': 0.11, 'C': 0.15, 'D': 0.17, 'E': 0.19, 'F': 0.21,
        'G': 0.09, 'H': 0.20, 'I': 0.17, 'K': 0.19, 'L': 0.17,
        'M': 0.19, 'N': 0.17, 'P': 0.15, 'Q': 0.19, 'R': 0.22,
        'S': 0.13, 'T': 0.15, 'V': 0.15, 'W': 0.25, 'Y': 0.23, 'X': 0.17
    }
}

def encode_sequence(sequence: str) -> List[int]:
    """Encode amino acid sequence to integer indices"""
    return [AA_TO_IDX.get(aa, 0) for aa in sequence.upper()]


def extract_physicochemical_features(sequence: str) -> np.ndarray:
    """
    Extract physicochemical features from sequence:
    - Mean, std, min, max for each property
    - Sequence composition (AA frequencies)
    - C-terminal and N-terminal features
    """
    seq = sequence.upper()
    features = []

    # For each physicochemical property
    for prop_name, prop_dict in AA_PROPERTIES.items():
        values = [prop_dict.get(aa, 0) for aa in seq]
        features.extend([
            np.mean(values),
            np.std(values),
            np.min(values),
            np.max(values)
        ])

    # Amino acid composition (20 features)
    total = len(seq)
    for aa in AMINO_ACIDS:
        features.append(seq.count(aa) / total if total > 0 else 0)

    # N-terminal and C-terminal features (first/last 10 residues)
    n_term = seq[:10].ljust(10, 'X')
    c_term = seq[-10:].rjust(10, 'X')

    for aa in n_term + c_term:
        for prop_name, prop_dict in AA_PROPERTIES.items():
            features.append(prop_dict.get(aa, 0))

    return np.array(features, dtype=np.float32)

In [ ]:
def generate_kmers(sequence: str, k: int) -> List[str]:
    """Generate all k-mers from a sequence"""
    sequence = sequence.upper()
    return [sequence[i:i+k] for i in range(len(sequence) - k + 1)]


def get_top_kmers(sequences: List[str], k: int, top_n: int = 150) -> List[str]:
    """Get top N most frequent k-mers (increased from 100 to 150)"""
    kmer_counts = {}

    for seq in sequences:
        kmers = generate_kmers(seq, k)
        for kmer in kmers:
            kmer_counts[kmer] = kmer_counts.get(kmer, 0) + 1

    sorted_kmers = sorted(kmer_counts.items(), key=lambda x: x[1], reverse=True)
    return [kmer for kmer, count in sorted_kmers[:top_n]]


def extract_kmer_features_tfidf(sequence: str, kmer_list: List[str], k: int,
                                 idf_weights: Optional[Dict[str, float]] = None) -> np.ndarray:
    """
    Extract TF-IDF k-mer features instead of binary presence/absence
    TF = frequency in sequence, IDF = inverse document frequency
    """
    seq_kmers = generate_kmers(sequence, k)
    kmer_counts = {}
    for kmer in seq_kmers:
        kmer_counts[kmer] = kmer_counts.get(kmer, 0) + 1

    features = []
    total_kmers = len(seq_kmers) if len(seq_kmers) > 0 else 1

    for kmer in kmer_list:
        tf = kmer_counts.get(kmer, 0) / total_kmers
        if idf_weights and kmer in idf_weights:
            features.append(tf * idf_weights[kmer])
        else:
            features.append(tf)

    return np.array(features, dtype=np.float32)


def calculate_idf_weights(sequences: List[str], kmer_list: List[str], k: int) -> Dict[str, float]:
    """Calculate IDF weights for k-mers"""
    doc_counts = {kmer: 0 for kmer in kmer_list}

    for seq in sequences:
        seq_kmers = set(generate_kmers(seq, k))
        for kmer in kmer_list:
            if kmer in seq_kmers:
                doc_counts[kmer] += 1

    n_docs = len(sequences)
    idf_weights = {}
    for kmer, count in doc_counts.items():
        idf_weights[kmer] = np.log(n_docs / (count + 1))

    return idf_weights


# ============================================================================
# 3. DATA AUGMENTATION
# ============================================================================

def augment_sequence(sequence: str, mutation_rate: float = 0.02) -> str:
    """
    Augment sequence with random mutations
    - Conservative mutations (similar physicochemical properties)
    """
    seq_list = list(sequence.upper())
    n_mutations = max(1, int(len(seq_list) * mutation_rate))

    # Conservative mutation groups
    conservative_groups = [
        ['A', 'G', 'S', 'T'],  # Small, polar
        ['D', 'E'],  # Acidic
        ['K', 'R', 'H'],  # Basic
        ['I', 'L', 'V', 'M'],  # Hydrophobic
        ['F', 'Y', 'W'],  # Aromatic
        ['N', 'Q'],  # Amide
    ]

    for _ in range(n_mutations):
        pos = random.randint(0, len(seq_list) - 1)
        original_aa = seq_list[pos]

        # Find conservative replacement
        for group in conservative_groups:
            if original_aa in group and len(group) > 1:
                replacements = [aa for aa in group if aa != original_aa]
                seq_list[pos] = random.choice(replacements)
                break

    return ''.join(seq_list)

In [ ]:
class MultiHeadAttention(nn.Module):
    """Multi-head self-attention mechanism"""

    def __init__(self, embed_dim: int, num_heads: int = 8, dropout: float = 0.1):
        super().__init__()
        assert embed_dim % num_heads == 0

        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads

        self.q_linear = nn.Linear(embed_dim, embed_dim)
        self.k_linear = nn.Linear(embed_dim, embed_dim)
        self.v_linear = nn.Linear(embed_dim, embed_dim)
        self.out_linear = nn.Linear(embed_dim, embed_dim)

        self.dropout = nn.Dropout(dropout)
        self.layer_norm = nn.LayerNorm(embed_dim)

    def forward(self, x):
        # x: (batch, embed_dim)
        # Add sequence dimension
        x = x.unsqueeze(1)  # (batch, 1, embed_dim)

        batch_size = x.size(0)

        # Linear projections
        Q = self.q_linear(x).view(batch_size, 1, self.num_heads, self.head_dim).transpose(1, 2)
        K = self.k_linear(x).view(batch_size, 1, self.num_heads, self.head_dim).transpose(1, 2)
        V = self.v_linear(x).view(batch_size, 1, self.num_heads, self.head_dim).transpose(1, 2)

        # Attention scores
        scores = torch.matmul(Q, K.transpose(-2, -1)) / np.sqrt(self.head_dim)
        attn_weights = F.softmax(scores, dim=-1)
        attn_weights = self.dropout(attn_weights)

        # Apply attention to values
        attn_output = torch.matmul(attn_weights, V)
        attn_output = attn_output.transpose(1, 2).contiguous().view(batch_size, 1, self.embed_dim)

        # Output projection
        output = self.out_linear(attn_output)
        output = output.squeeze(1)  # (batch, embed_dim)

        # Residual connection and layer norm
        output = self.layer_norm(x.squeeze(1) + self.dropout(output))

        return output


In [ ]:
class FocalLoss(nn.Module):
    """
    Focal Loss for addressing class imbalance
    Focuses training on hard examples
    """
    def __init__(self, alpha: float = 0.25, gamma: float = 2.0):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma

    def forward(self, inputs, targets):
        ce_loss = F.cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-ce_loss)
        focal_loss = self.alpha * (1 - pt) ** self.gamma * ce_loss
        return focal_loss.mean()


class ImprovedBacteriocinClassifier(nn.Module):
    """
    Improved Bacteriocin Classifier with:
    - Multi-head attention
    - Residual connections
    - Batch normalization
    - Deeper architecture
    """

    def __init__(self, input_dim: int, hidden_dims: List[int] = [512, 256, 128, 64]):
        super().__init__()

        self.input_projection = nn.Sequential(
            nn.Linear(input_dim, hidden_dims[0]),
            nn.BatchNorm1d(hidden_dims[0]),
            nn.ReLU(),
            nn.Dropout(0.3)
        )

        # Multi-head attention
        self.attention = MultiHeadAttention(hidden_dims[0], num_heads=8, dropout=0.2)

        # Deep residual blocks
        self.blocks = nn.ModuleList()
        for i in range(len(hidden_dims) - 1):
            self.blocks.append(
                nn.Sequential(
                    nn.Linear(hidden_dims[i], hidden_dims[i+1]),
                    nn.BatchNorm1d(hidden_dims[i+1]),
                    nn.ReLU(),
                    nn.Dropout(0.3)
                )
            )

        # Output layer
        self.output = nn.Sequential(
            nn.Linear(hidden_dims[-1], 32),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(32, 2)
        )

    def forward(self, x):
        # Input projection
        x = self.input_projection(x)

        # Self-attention
        x = self.attention(x)

        # Deep blocks
        for block in self.blocks:
            x = block(x)

        # Output
        x = self.output(x)
        return x

In [ ]:
class ImprovedBacteriocinDataset(Dataset):
    """Dataset with optional augmentation"""

    def __init__(self, features: np.ndarray, labels: np.ndarray,
                 sequences: Optional[List[str]] = None,
                 kmer_lists: Optional[Dict] = None,
                 idf_weights: Optional[Dict] = None,
                 augment: bool = False):
        self.features = torch.FloatTensor(features)
        self.labels = torch.LongTensor(labels)
        self.sequences = sequences
        self.kmer_lists = kmer_lists
        self.idf_weights = idf_weights
        self.augment = augment

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        features = self.features[idx]
        label = self.labels[idx]

        # Optional: augment on-the-fly (only for training)
        if self.augment and self.sequences and random.random() < 0.3:
            # Re-extract features from augmented sequence
            aug_seq = augment_sequence(self.sequences[idx])
            # Note: In practice, this would need the full feature extraction pipeline
            # For now, we'll just use original features
            pass

        return features, label

In [ ]:
def train_epoch_improved(model, dataloader, criterion, optimizer, device,
                         clip_grad: float = 1.0):
    """Train with gradient clipping"""
    model.train()
    total_loss = 0

    for features, labels in dataloader:
        features, labels = features.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()

        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip_grad)

        optimizer.step()
        total_loss += loss.item()

    return total_loss / len(dataloader)


def evaluate_improved(model, dataloader, criterion, device):
    """Enhanced evaluation with more metrics"""
    model.eval()
    total_loss = 0
    all_preds = []
    all_labels = []
    all_probs = []

    with torch.no_grad():
        for features, labels in dataloader:
            features, labels = features.to(device), labels.to(device)

            outputs = model(features)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            probs = F.softmax(outputs, dim=1)
            preds = torch.argmax(outputs, dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs[:, 1].cpu().numpy())

    avg_loss = total_loss / len(dataloader)
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='binary', zero_division=0)
    recall = recall_score(all_labels, all_preds, average='binary', zero_division=0)
    f1 = f1_score(all_labels, all_preds, average='binary', zero_division=0)

    try:
        auc = roc_auc_score(all_labels, all_probs)
    except:
        auc = 0.0

    return avg_loss, accuracy, precision, recall, f1, auc, all_preds, all_labels


In [ ]:
def kfold_cross_validation_improved(features: np.ndarray,
                                    labels: np.ndarray,
                                    sequences: Optional[List[str]] = None,
                                    k: int = 30,  # Reduced from 30 for faster training
                                    epochs: int = 100,
                                    batch_size: int = 64,  # Increased from 40
                                    learning_rate: float = 1e-4,  # Higher initial LR
                                    device: str = 'cuda' if torch.cuda.is_available() else 'cpu',
                                    patience: int = 15):
    """
    Improved k-fold cross validation with:
    - Stratified splits
    - Early stopping
    - Cosine annealing LR schedule
    - Focal loss
    """

    # Use StratifiedKFold to maintain class balance
    kfold = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)
    fold_results = []

    for fold, (train_idx, val_idx) in enumerate(kfold.split(features, labels)):
        print(f"\n{'='*60}")
        print(f"Fold {fold + 1}/{k}")
        print(f"{'='*60}")

        # Split data
        X_train, X_val = features[train_idx], features[val_idx]
        y_train, y_val = labels[train_idx], labels[val_idx]

        # Create datasets
        train_dataset = ImprovedBacteriocinDataset(X_train, y_train)
        val_dataset = ImprovedBacteriocinDataset(X_val, y_val)

        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

        # Initialize model
        model = ImprovedBacteriocinClassifier(
            input_dim=features.shape[1],
            hidden_dims=[512, 256, 128, 64]
        ).to(device)

        # Focal loss for better handling of hard examples
        criterion = FocalLoss(alpha=0.25, gamma=2.0)
        optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=0.01)

        # Cosine annealing learning rate scheduler
        scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
            optimizer, T_0=10, T_mult=2, eta_min=1e-6
        )

        # Training history
        train_losses = []
        val_losses = []
        val_accuracies = []
        val_f1_scores = []

        best_f1 = 0
        patience_counter = 0
        best_model_state = None

        # Training loop with early stopping
        for epoch in range(epochs):
            train_loss = train_epoch_improved(model, train_loader, criterion, optimizer, device)
            val_loss, val_acc, val_prec, val_rec, val_f1, val_auc, _, _ = evaluate_improved(
                model, val_loader, criterion, device
            )

            train_losses.append(train_loss)
            val_losses.append(val_loss)
            val_accuracies.append(val_acc)
            val_f1_scores.append(val_f1)

            scheduler.step()

            # Early stopping based on F1 score
            if val_f1 > best_f1:
                best_f1 = val_f1
                patience_counter = 0
                best_model_state = model.state_dict().copy()
            else:
                patience_counter += 1

            if (epoch + 1) % 10 == 0:
                print(f"Epoch {epoch+1}/{epochs} - "
                      f"Train Loss: {train_loss:.4f}, "
                      f"Val Loss: {val_loss:.4f}, "
                      f"Val Acc: {val_acc:.4f}, "
                      f"Val F1: {val_f1:.4f}, "
                      f"Val AUC: {val_auc:.4f}")

            if patience_counter >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                break

        # Load best model
        if best_model_state:
            model.load_state_dict(best_model_state)

        # Final evaluation
        final_loss, final_acc, final_prec, final_rec, final_f1, final_auc, preds, true_labels = evaluate_improved(
            model, val_loader, criterion, device
        )

        fold_results.append({
            'fold': fold + 1,
            'loss': final_loss,
            'accuracy': final_acc * 100,
            'precision': final_prec,
            'recall': final_rec,
            'f1_score': final_f1,
            'auc': final_auc,
            'train_losses': train_losses,
            'val_losses': val_losses,
            'val_accuracies': val_accuracies,
            'val_f1_scores': val_f1_scores,
            'predictions': preds,
            'true_labels': true_labels,
            'model_state': model.state_dict()
        })

        print(f"\nFold {fold + 1} Results:")
        print(f"Loss: {final_loss:.4f}")
        print(f"Accuracy: {final_acc * 100:.2f}%")
        print(f"Precision: {final_prec:.4f}")
        print(f"Recall: {final_rec:.4f}")
        print(f"F1 Score: {final_f1:.4f}")
        print(f"AUC: {final_auc:.4f}")

    return fold_results

In [ ]:
def ensemble_predict(models: List[nn.Module], features: np.ndarray, device: str) -> np.ndarray:
    """
    Ensemble prediction using multiple models
    Returns averaged probabilities
    """
    all_probs = []

    features_tensor = torch.FloatTensor(features).to(device)

    for model in models:
        model.eval()
        with torch.no_grad():
            outputs = model(features_tensor)
            probs = F.softmax(outputs, dim=1)
            all_probs.append(probs.cpu().numpy())

    # Average probabilities
    avg_probs = np.mean(all_probs, axis=0)
    predictions = np.argmax(avg_probs, axis=1)

    return predictions, avg_probs

In [ ]:
def save_training_data(fold_results: List[Dict], best_fold_idx: int, filename: str = 'training_data.pkl'):
    """Save training data for later visualization"""
    import pickle

    training_data = {
        'fold_results': fold_results,
        'best_fold_idx': best_fold_idx,
        'timestamp': pd.Timestamp.now().isoformat(),
        'num_folds': len(fold_results),
        'summary_metrics': {
            'mean_accuracy': np.mean([r['accuracy'] for r in fold_results]),
            'mean_precision': np.mean([r['precision'] for r in fold_results]),
            'mean_recall': np.mean([r['recall'] for r in fold_results]),
            'mean_f1': np.mean([r['f1_score'] for r in fold_results]),
            'mean_auc': np.mean([r['auc'] for r in fold_results]),
            'std_accuracy': np.std([r['accuracy'] for r in fold_results]),
        }
    }

    with open(filename, 'wb') as f:
        pickle.dump(training_data, f)

    print(f"\n✓ Training data saved as '{filename}'")
    print(f"  You can regenerate/edit visualizations without retraining by loading this file")
    return training_data


def plot_improved_results(fold_results: List[Dict], best_fold_idx: int):
    """Plot training curves and confusion matrix as separate PNG files"""
    import os
    best_fold = fold_results[best_fold_idx]
    epochs = range(1, len(best_fold['val_accuracies']) + 1)

    # Create output directory
    os.makedirs('plots', exist_ok=True)

    print("\n" + "="*70)
    print("GENERATING VISUALIZATIONS")
    print("="*70)

    # 1. Accuracy curve
    plt.figure(figsize=(10, 6))
    plt.plot(epochs, [acc * 100 for acc in best_fold['val_accuracies']],
             'b-', linewidth=2.5, label='Validation Accuracy', marker='o', markersize=4)
    plt.xlabel('Epoch', fontsize=14)
    plt.ylabel('Accuracy (%)', fontsize=14)
    plt.title(f'Validation Accuracy - Fold {best_fold["fold"]} (Best: {best_fold["accuracy"]:.2f}%)',
              fontsize=16, fontweight='bold')
    plt.legend(fontsize=12)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig('plots/accuracy_curve.png', dpi=300, bbox_inches='tight')
    print("✓ Saved: plots/accuracy_curve.png")
    plt.close()

    # 2. Loss curves
    plt.figure(figsize=(10, 6))
    plt.plot(epochs, best_fold['train_losses'], 'r-', linewidth=2.5,
             label='Training Loss', marker='s', markersize=4)
    plt.plot(epochs, best_fold['val_losses'], 'b-', linewidth=2.5,
             label='Validation Loss', marker='o', markersize=4)
    plt.xlabel('Epoch', fontsize=14)
    plt.ylabel('Loss', fontsize=14)
    plt.title(f'Training Curves - Fold {best_fold["fold"]}',
              fontsize=16, fontweight='bold')
    plt.legend(fontsize=12)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig('plots/loss_curves.png', dpi=300, bbox_inches='tight')
    print("✓ Saved: plots/loss_curves.png")
    plt.close()

    # 3. F1 Score curve
    plt.figure(figsize=(10, 6))
    plt.plot(epochs, best_fold['val_f1_scores'], 'g-', linewidth=2.5,
             label='F1 Score', marker='^', markersize=4)
    plt.xlabel('Epoch', fontsize=14)
    plt.ylabel('F1 Score', fontsize=14)
    plt.title(f'F1 Score - Fold {best_fold["fold"]} (Best: {best_fold["f1_score"]:.4f})',
              fontsize=16, fontweight='bold')
    plt.legend(fontsize=12)
    plt.grid(True, alpha=0.3)
    plt.ylim(0, 1.05)
    plt.tight_layout()
    plt.savefig('plots/f1_score_curve.png', dpi=300, bbox_inches='tight')
    print("✓ Saved: plots/f1_score_curve.png")
    plt.close()

    # 4. Confusion Matrix (normalized)
    cm = confusion_matrix(best_fold['true_labels'], best_fold['predictions'])
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    plt.figure(figsize=(8, 7))
    sns.heatmap(cm_normalized, annot=True, fmt='.2%', cmap='Blues',
                xticklabels=['Non-BacLAB', 'BacLAB'],
                yticklabels=['Non-BacLAB', 'BacLAB'],
                cbar_kws={'label': 'Percentage'})
    plt.xlabel('Predicted', fontsize=14)
    plt.ylabel('True', fontsize=14)
    plt.title(f'Confusion Matrix - Fold {best_fold["fold"]} (Normalized)',
              fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.savefig('plots/confusion_matrix_normalized.png', dpi=300, bbox_inches='tight')
    print("✓ Saved: plots/confusion_matrix_normalized.png")
    plt.close()

    # 5. Confusion Matrix (raw counts)
    plt.figure(figsize=(8, 7))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Oranges',
                xticklabels=['Non-BacLAB', 'BacLAB'],
                yticklabels=['Non-BacLAB', 'BacLAB'],
                cbar_kws={'label': 'Count'})
    plt.xlabel('Predicted', fontsize=14)
    plt.ylabel('True', fontsize=14)
    plt.title(f'Confusion Matrix - Fold {best_fold["fold"]} (Counts)',
              fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.savefig('plots/confusion_matrix_counts.png', dpi=300, bbox_inches='tight')
    print("✓ Saved: plots/confusion_matrix_counts.png")
    plt.close()

    # 6. All metrics comparison across folds
    plt.figure(figsize=(12, 6))
    fold_nums = [r['fold'] for r in fold_results]
    accuracies = [r['accuracy'] for r in fold_results]
    precisions = [r['precision'] * 100 for r in fold_results]
    recalls = [r['recall'] * 100 for r in fold_results]
    f1_scores = [r['f1_score'] * 100 for r in fold_results]

    x = np.arange(len(fold_nums))
    width = 0.2

    plt.bar(x - 1.5*width, accuracies, width, label='Accuracy', alpha=0.8)
    plt.bar(x - 0.5*width, precisions, width, label='Precision', alpha=0.8)
    plt.bar(x + 0.5*width, recalls, width, label='Recall', alpha=0.8)
    plt.bar(x + 1.5*width, f1_scores, width, label='F1 Score', alpha=0.8)

    plt.xlabel('Fold', fontsize=14)
    plt.ylabel('Score (%)', fontsize=14)
    plt.title('Performance Metrics Across All Folds', fontsize=16, fontweight='bold')
    plt.xticks(x, fold_nums)
    plt.legend(fontsize=11)
    plt.grid(True, alpha=0.3, axis='y')
    plt.ylim(85, 100)
    plt.tight_layout()
    plt.savefig('plots/all_folds_comparison.png', dpi=300, bbox_inches='tight')
    print("✓ Saved: plots/all_folds_comparison.png")
    plt.close()

    # 7. Box plot of metrics across folds
    plt.figure(figsize=(10, 6))
    metrics_data = [
        [r['accuracy'] for r in fold_results],
        [r['precision'] * 100 for r in fold_results],
        [r['recall'] * 100 for r in fold_results],
        [r['f1_score'] * 100 for r in fold_results],
        [r['auc'] * 100 for r in fold_results]
    ]

    bp = plt.boxplot(metrics_data, labels=['Accuracy', 'Precision', 'Recall', 'F1 Score', 'AUC'],
                     patch_artist=True, showmeans=True)

    colors = ['lightblue', 'lightgreen', 'lightyellow', 'lightcoral', 'plum']
    for patch, color in zip(bp['boxes'], colors):
        patch.set_facecolor(color)

    plt.ylabel('Score (%)', fontsize=14)
    plt.title('Distribution of Metrics Across All Folds', fontsize=16, fontweight='bold')
    plt.grid(True, alpha=0.3, axis='y')
    plt.ylim(85, 100)
    plt.tight_layout()
    plt.savefig('plots/metrics_distribution.png', dpi=300, bbox_inches='tight')
    print("✓ Saved: plots/metrics_distribution.png")
    plt.close()

    print("\n✓ All visualizations saved in 'plots/' directory")
    print("  Files created:")
    print("    - accuracy_curve.png")
    print("    - loss_curves.png")
    print("    - f1_score_curve.png")
    print("    - confusion_matrix_normalized.png")
    print("    - confusion_matrix_counts.png")
    print("    - all_folds_comparison.png")
    print("    - metrics_distribution.png")


def main_improved():
    """
    Improved main pipeline with state-of-the-art techniques
    """

    print("="*70)
    print("BacLABNet v2.0: Improved Bacteriocin Classification")
    print("="*70)

    # Set random seeds for reproducibility
    torch.manual_seed(42)
    np.random.seed(42)
    random.seed(42)

    # 1. Load data
    print("\n[1/6] Loading data...")
    df = pd.read_csv('data_BacLAB_and_nonBacLAB.csv',
                     header=None,
                     names=['ID', 'Species', 'Sequence', 'Label', 'Empty'])

    sequences = df['Sequence'].tolist()
    labels = df['Label'].values

    print(f"Total sequences: {len(sequences)}")
    print(f"BacLAB: {sum(labels)}, Non-BacLAB: {len(labels) - sum(labels)}")

    # 2. Extract improved k-mer features (with TF-IDF)
    print("\n[2/6] Extracting improved k-mer features (TF-IDF)...")

    try:
        kmers_df = pd.read_csv('List_kmers.csv')
        kmers_5 = kmers_df['5-mers'].dropna().tolist()[:150]  # Top 150 instead of 100
        kmers_7 = kmers_df['7-mers'].dropna().tolist()[:150]
        print("Loaded pre-computed k-mers")
    except:
        print("Computing k-mers...")
        baclab_sequences = [seq for seq, lbl in zip(sequences, labels) if lbl == 1]
        kmers_5 = get_top_kmers(baclab_sequences, k=5, top_n=150)
        kmers_7 = get_top_kmers(baclab_sequences, k=7, top_n=150)

    # Calculate IDF weights
    print("Calculating TF-IDF weights...")
    idf_5 = calculate_idf_weights(sequences, kmers_5, 5)
    idf_7 = calculate_idf_weights(sequences, kmers_7, 7)

    features_5 = np.array([extract_kmer_features_tfidf(seq, kmers_5, 5, idf_5) for seq in sequences])
    features_7 = np.array([extract_kmer_features_tfidf(seq, kmers_7, 7, idf_7) for seq in sequences])

    print(f"5-mer TF-IDF features: {features_5.shape}")
    print(f"7-mer TF-IDF features: {features_7.shape}")

    # 3. Extract physicochemical features
    print("\n[3/6] Extracting physicochemical features...")
    start_time = time.time()
    physchem_features = np.array([extract_physicochemical_features(seq) for seq in sequences])
    print(f"Physicochemical features: {physchem_features.shape}")
    print(f"Time: {time.time() - start_time:.2f} seconds")

    # 4. Load pre-computed embeddings (or use simpler features if not available)
    print("\n[4/6] Loading embedding features...")
    embedding_features = None

    # Try ESM-2 embeddings first (best quality), then fall back to GRU embeddings
    try:
        embedding_features = np.load('esm2_embeddings.npy')
        print(f"✓ Loaded ESM-2 embeddings: {embedding_features.shape}")
        print("  (Using state-of-the-art ESM-2 protein language model embeddings)")
    except:
        try:
            embedding_features = np.load('embeddings.npy')
            print(f"✓ Loaded embeddings: {embedding_features.shape}")
            print("  (Using GRU embeddings - for best results, run esm2_embeddings.py)")
        except:
            print("⚠ No pre-computed embeddings found. Using only k-mers and physicochemical features.")
            print("  → For best results (+5-8% accuracy), run: python esm2_embeddings.py")
            embedding_features = np.zeros((len(sequences), 0))

    # 5. Concatenate all features
    print("\n[5/6] Concatenating features...")
    if embedding_features.shape[1] > 0:
        features = np.concatenate([features_5, features_7, physchem_features, embedding_features], axis=1)
    else:
        features = np.concatenate([features_5, features_7, physchem_features], axis=1)

    print(f"Final features shape: {features.shape}")
    print(f"Feature breakdown:")
    print(f"  - 5-mer TF-IDF: {features_5.shape[1]}")
    print(f"  - 7-mer TF-IDF: {features_7.shape[1]}")
    print(f"  - Physicochemical: {physchem_features.shape[1]}")
    if embedding_features.shape[1] > 0:
        print(f"  - Embeddings: {embedding_features.shape[1]}")
    else:
        print(f"  - Embeddings: 0 (not used)")
    print(f"  - Total: {features.shape[1]}")

    # 6. Train with improved k-fold cross-validation
    print("\n[6/6] Starting improved k-fold cross-validation...")
    print("Improvements:")
    print("  ✓ Transformer architecture with multi-head attention")
    print("  ✓ Focal loss for better hard example learning")
    print("  ✓ Cosine annealing learning rate schedule")
    print("  ✓ Early stopping with patience=15")
    print("  ✓ Gradient clipping")
    print("  ✓ Deeper network (512→256→128→64→32→2)")

    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"  Device: {device.upper()}")

    fold_results = kfold_cross_validation_improved(
        features=features,
        labels=labels,
        sequences=sequences,
        k=30,  # Using 30 folds to match original paper
        epochs=100,
        batch_size=64,
        learning_rate=1e-4,
        device=device,
        patience=15
    )

    # 7. Analyze results
    print("\n" + "="*70)
    print("IMPROVED MODEL RESULTS")
    print("="*70)

    results_df = pd.DataFrame([{
        'Fold': r['fold'],
        'Loss': r['loss'],
        'Accuracy (%)': r['accuracy'],
        'Precision': r['precision'],
        'Recall': r['recall'],
        'F1 Score': r['f1_score'],
        'AUC': r['auc']
    } for r in fold_results])

    print(results_df.to_string(index=False))

    print("\n" + "="*70)
    print("AVERAGE METRICS")
    print("="*70)
    print(f"Loss: {results_df['Loss'].mean():.4f}")
    print(f"Accuracy: {results_df['Accuracy (%)'].mean():.2f}%")
    print(f"Precision: {results_df['Precision'].mean():.4f}")
    print(f"Recall: {results_df['Recall'].mean():.4f}")
    print(f"F1 Score: {results_df['F1 Score'].mean():.4f}")
    print(f"AUC: {results_df['AUC'].mean():.4f}")

    print("\n" + "="*70)
    print("COMPARISON WITH ORIGINAL")
    print("="*70)
    print(f"Original Paper: 90.14% accuracy")
    print(f"First Implementation: 85.04% accuracy")
    print(f"Improved Model: {results_df['Accuracy (%)'].mean():.2f}% accuracy")
    print(f"Improvement: {results_df['Accuracy (%)'].mean() - 90.14:+.2f}%")

    # Find best fold
    best_fold_idx = results_df['F1 Score'].idxmax()
    best_fold = fold_results[best_fold_idx]

    print("\n" + "="*70)
    print(f"BEST FOLD: Fold {best_fold['fold']}")
    print("="*70)
    print(f"Accuracy: {best_fold['accuracy']:.2f}%")
    print(f"Precision: {best_fold['precision']:.4f}")
    print(f"Recall: {best_fold['recall']:.4f}")
    print(f"F1 Score: {best_fold['f1_score']:.4f}")
    print(f"AUC: {best_fold['auc']:.4f}")

    # Save best model
    torch.save(best_fold['model_state'], 'best_model_improved.pt')
    print(f"\n✓ Best model saved as 'best_model_improved.pt'")

    # Save training data for later visualization
    save_training_data(fold_results, best_fold_idx, 'training_data.pkl')

    # Plot comparison
    plot_improved_results(fold_results, best_fold_idx)

    print("\n" + "="*70)
    print("PIPELINE COMPLETE!")
    print("="*70)

    return fold_results, results_df


if __name__ == "__main__":
    main_improved()

BacLABNet v2.0: Improved Bacteriocin Classification

[1/6] Loading data...
Total sequences: 49964
BacLAB: 24964, Non-BacLAB: 25000

[2/6] Extracting improved k-mer features (TF-IDF)...
Computing k-mers...
Calculating TF-IDF weights...
5-mer TF-IDF features: (49964, 150)
7-mer TF-IDF features: (49964, 150)

[3/6] Extracting physicochemical features...
Physicochemical features: (49964, 116)
Time: 34.49 seconds

[4/6] Loading embedding features...
✓ Loaded ESM-2 embeddings: (49964, 480)
  (Using state-of-the-art ESM-2 protein language model embeddings)

[5/6] Concatenating features...
Final features shape: (49964, 896)
Feature breakdown:
  - 5-mer TF-IDF: 150
  - 7-mer TF-IDF: 150
  - Physicochemical: 116
  - Embeddings: 480
  - Total: 896

[6/6] Starting improved k-fold cross-validation...
Improvements:
  ✓ Transformer architecture with multi-head attention
  ✓ Focal loss for better hard example learning
  ✓ Cosine annealing learning rate schedule
  ✓ Early stopping with patience=15
  ✓ 

/tmp/ipython-input-3731764925.py:160: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  bp = plt.boxplot(metrics_data, labels=['Accuracy', 'Precision', 'Recall', 'F1 Score', 'AUC'],


✓ Saved: plots/metrics_distribution.png

✓ All visualizations saved in 'plots/' directory
  Files created:
    - accuracy_curve.png
    - loss_curves.png
    - f1_score_curve.png
    - confusion_matrix_normalized.png
    - confusion_matrix_counts.png
    - all_folds_comparison.png
    - metrics_distribution.png

PIPELINE COMPLETE!
